In [1]:

import sys
sys.path.insert(1, '../py')
from FRVRS import nu, fu
nu.delete_ipynb_checkpoint_folders()

In [2]:

# Set up notebook
%pprint
import sys
sys.path.insert(1, '../tests')

Pretty printing has been turned OFF


In [3]:

import unittest

In [4]:

# Create another test suite
suite = unittest.TestSuite()

# Add all test cases from the class to the suite
from test_frvrs_utils_other_functions import TestFormatTimedelta
suite.addTest(unittest.makeSuite(TestFormatTimedelta))
from test_frvrs_utils_pandas_functions import TestGetStatistics, TestGetElevensDataFrame
suite.addTest(unittest.makeSuite(TestGetStatistics))
suite.addTest(unittest.makeSuite(TestGetElevensDataFrame))
from test_frvrs_utils_patient_functions import TestGetMaxSalt, TestGetLastTag, TestIsPatientHemorrhagingFunction, TestGetTimeToHemorrhageControl
suite.addTest(unittest.makeSuite(TestGetMaxSalt))
suite.addTest(unittest.makeSuite(TestGetLastTag))
suite.addTest(unittest.makeSuite(TestIsPatientHemorrhagingFunction))
suite.addTest(unittest.makeSuite(TestGetTimeToHemorrhageControl))

# Run the test suite
runner = unittest.TextTestRunner()
runner.run(suite)

.................
----------------------------------------------------------------------
Ran 17 tests in 34.517s

OK


<unittest.runner.TextTestResult run=17 errors=0 failures=0>

In [5]:

# Create another test suite
suite = unittest.TestSuite()

# Add all test cases from the class to the suite
from test_frvrs_utils_scene_functions import TestGetTriageTime, TestGetPercentHemorrhageControlled, TestGetTimeToLastHemorrhageControlled
suite.addTest(unittest.makeSuite(TestGetTriageTime))
suite.addTest(unittest.makeSuite(TestGetPercentHemorrhageControlled))
suite.addTest(unittest.makeSuite(TestGetTimeToLastHemorrhageControlled))

# Run the test suite
runner = unittest.TextTestRunner()
runner.run(suite)

.....EEEEEE
ERROR: test_multiple_patients_hemorrhage (test_frvrs_utils_scene_functions.TestGetTimeToLastHemorrhageControlled)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/tests/test_frvrs_utils_scene_functions.py", line 984, in test_multiple_patients_hemorrhage
    self.patch_get_time_to_hemorrhage_control(side_effect=get_time_to_hemorrhage_control)
  File "/mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/tests/test_frvrs_utils_scene_functions.py", line 1004, in patch_get_time_to_hemorrhage_control
    return patcher.start()
  File "/home/dbabbitt/anaconda3/envs/itm_analysis_reporting/lib/python3.7/unittest/mock.py", line 1442, in start
    result = self.__enter__()
  File "/home/dbabbitt/anaconda3/envs/itm_analysis_reporting/lib/python3.7/unittest/mock.py", line 1307, in __enter__
    original, local = self.get_original()
  File "/home/d

<unittest.runner.TextTestResult run=8 errors=6 failures=0>


# Create some good Mock Data

In [4]:

# Get the standard data frames
if ('../py' not in sys.path): sys.path.insert(1, '../py')
from FRVRS import fu, nu
data_frames_list = nu.load_data_frames(
    verbose=False, first_responder_master_registry_df='', first_responder_master_registry_file_stats_df='', first_responder_master_registry_scene_stats_df=''
)
triage_paper_df = data_frames_list['first_responder_master_registry_df']
print(triage_paper_df.shape)
file_stats_df = data_frames_list['first_responder_master_registry_file_stats_df']
print(file_stats_df.shape)
scene_stats_df = data_frames_list['first_responder_master_registry_scene_stats_df']
print(scene_stats_df.shape)
elevens_df = fu.get_elevens_data_frame(
    triage_paper_df, file_stats_df, scene_stats_df, needed_columns=['scene_type', 'is_scene_aborted', 'is_a_one_triage_file', 'responder_category']
)
print(elevens_df.shape)

(401177, 111)
(332, 19)
(486, 48)
(399048, 115)


In [15]:

# Get all the non-hemorrhaging procedures
from random import choices
import pandas as pd

nons_list = sorted(set(fu.injury_required_procedure_order).difference(set(fu.hemorrhage_control_procedures_list)))

# Create a list 10 elements long using choices with replacement
ten_element_list = choices(nons_list, k=10)

scene_df = DataFrame({
    'action_tick': [
        53712, 111438, 173005, 201999, 235440,
        249305, 296810, 329527, 363492, 442375
    ],
    'patient_id': [
        'Gloria_2 Root', 'Lily_1 Root', 'Lily_1 Root', 'Mike_3 Root', 'Helga_0 Root',
        'Helga_0 Root', 'Mike_6 Root', 'Mike_9 Root', 'Mike_9 Root', 'Mike_4 Root'
    ],
    'injury_id': [
        'L Shin Amputation', 'R Thigh Laceration', 'R Bicep Puncture', 'R Wrist Amputation', 'L Stomach Puncture',
        'L Thigh Puncture', 'R Chest Collapse', 'L Side Puncture', 'L Thigh Puncture', 'R Palm Laceration'
    ],
    'injury_required_procedure': ten_element_list,
    'injury_record_required_procedure': ten_element_list,
    'injury_treated_required_procedure': ten_element_list,
    'injury_treated_injury_treated_with_wrong_treatment': some_trues
})
pd.isna(fu.get_percent_hemorrhage_controlled(scene_df))

True

In [ ]:

# Find an injury where the required procedures don't match
columns_list = ['injury_record_required_procedure', 'injury_treated_required_procedure']
rows_list = []
for (session_uuid, scene_id, patient_id, injury_id), injury_df in elevens_df.groupby(fu.injury_groupby_columns):
    row_dict = {cn: eval(cn) for cn in fu.injury_groupby_columns}
    df = injury_df[columns_list].drop_duplicates()
    row_dict['row_count'] = df.shape[0]
    rows_list.append(row_dict)
df = DataFrame(rows_list)
df.sort_values('row_count', ascending=False)

In [ ]:

# Find an injury where the required procedures don't match
columns_list = ['injury_record_required_procedure', 'injury_treated_required_procedure']
rows_list = []
for (session_uuid, scene_id, patient_id, injury_id), injury_df in elevens_df.groupby(fu.injury_groupby_columns):
    row_dict = {cn: eval(cn) for cn in fu.injury_groupby_columns}
    df = injury_df[columns_list].drop_duplicates()
    if df.shape[0] == 1: display(df); break

In [ ]:

# Get the scenes with the highest hemorrhage control percentage
max_percent = scene_stats_df.percent_hemorrhage_controlled.max()
mask_series = (scene_stats_df.percent_hemorrhage_controlled == max_percent)
df = scene_stats_df[mask_series]
on_columns = sorted(set(elevens_df.columns).intersection(set(df.columns)))
merge_df = elevens_df.merge(df, on=on_columns, how='inner')

In [ ]:

# Find the scenes with wrong treatments in them
mask_series = (merge_df.injury_treated_injury_treated_with_wrong_treatment == True)
display(mask_series.value_counts(dropna=False))
for (session_uuid, scene_id), scene_df in merge_df[mask_series].groupby(fu.scene_groupby_columns):
    
    # Get the entire scene history
    scene_mask_series = True
    for cn in fu.scene_groupby_columns: scene_mask_series &= (merge_df[cn] == eval(cn))
    scene_df = merge_df[scene_mask_series]
    
    display(scene_df.injury_treated_injury_treated_with_wrong_treatment.value_counts(dropna=False))
    # if (scene_df.shape[0] >= 5): break

In [ ]:

mask_series = ~scene_df.injury_required_procedure.isnull()
print(scene_df[mask_series].shape)
scene_df[mask_series].injury_required_procedure.unique()

In [ ]:

mask_series = ~scene_df.injury_treated_injury_treated_with_wrong_treatment.isnull()
print(scene_df[mask_series].shape)
scene_df[mask_series].injury_treated_injury_treated_with_wrong_treatment.unique()

In [ ]:

# df = scene_df.sample(15)
# while not df.injury_treated_injury_treated_with_wrong_treatment.any(): df = scene_df.sample(15)
mask_series = ~scene_df.injury_treated_injury_treated_with_wrong_treatment.isnull()
df = scene_df[mask_series]
columns_list = [
    'injury_id', 'injury_required_procedure', 'injury_record_required_procedure', 'injury_treated_required_procedure',
    'injury_treated_injury_treated_with_wrong_treatment'
]
print(f"""
    def setUp(self):
        # Create a sample DataFrame
        self.scene_df = DataFrame({{
            'action_tick': {df.action_tick.tolist()},
            'patient_id': {df.patient_id.tolist()},""")
for cn in columns_list: print(f"""            '{cn}': {df[cn].tolist()},""")
print(f"""        }})""")

In [16]:

from pandas import DataFrame
from numpy import nan

# Create a sample DataFrame
procedures_list = [
    'tourniquet', 'tourniquet', 'tourniquet', 'tourniquet', 'woundpack',
    'tourniquet', 'decompress', 'woundpack', 'tourniquet', 'gauzePressure'
]
all_trues = [True] * len(procedures_list)
some_trues = [
    False, False, False, False, False,
    True, False, False, False, False
]
all_falses = [False] * len(procedures_list)
scene_df = DataFrame({
    'action_tick': [
        53712, 111438, 173005, 201999, 235440,
        249305, 296810, 329527, 363492, 442375
    ],
    'patient_id': [
        'Gloria_2 Root', 'Lily_1 Root', 'Lily_1 Root', 'Mike_3 Root', 'Helga_0 Root',
        'Helga_0 Root', 'Mike_6 Root', 'Mike_9 Root', 'Mike_9 Root', 'Mike_4 Root'
    ],
    'injury_id': [
        'L Shin Amputation', 'R Thigh Laceration', 'R Bicep Puncture', 'R Wrist Amputation', 'L Stomach Puncture',
        'L Thigh Puncture', 'R Chest Collapse', 'L Side Puncture', 'L Thigh Puncture', 'R Palm Laceration'
    ],
    'injury_required_procedure': procedures_list,
    'injury_record_required_procedure': procedures_list,
    'injury_treated_required_procedure': procedures_list,
    'injury_treated_injury_treated_with_wrong_treatment': some_trues
})
fu.get_percent_hemorrhage_controlled(scene_df, verbose=True)

Hemorrhage-related injuries that have been recorded or treated: 8
Injuries requiring hemorrhage control procedures: 8
Hemorrhage-related injuries that have been treated: 7
Percentage of hemorrhage controlled: 87.50%


,injury_required_procedure,injury_record_required_procedure,injury_treated_required_procedure,injury_treated_injury_treated_with_wrong_treatment
0,tourniquet,tourniquet,tourniquet,False
4,woundpack,woundpack,woundpack,False
5,tourniquet,tourniquet,tourniquet,True
6,decompress,decompress,decompress,False
9,gauzePressure,gauzePressure,gauzePressure,False


87.5

In [18]:

# Test the function with duplicate treatment entries for the same injury
duplicate_df = scene_df.append(scene_df.iloc[3])
fu.get_percent_hemorrhage_controlled(duplicate_df)

87.5

In [8]:

# Loop through each injury, examining its required procedures and wrong treatments
columns_list = [
    'patient_id', 'injury_id', 'injury_required_procedure', 'injury_record_required_procedure', 'injury_treated_required_procedure',
    'injury_treated_injury_treated_with_wrong_treatment'
]
for (patient_id, injury_id), injury_df in scene_df.groupby(['patient_id', 'injury_id']):
    print()
    for cn in columns_list: print(f'{cn}: {injury_df[cn].tolist()}')


patient_id: ['Gloria_2 Root']
injury_id: ['L Shin Amputation']
injury_required_procedure: ['tourniquet']
injury_record_required_procedure: ['tourniquet']
injury_treated_required_procedure: ['tourniquet']
injury_treated_injury_treated_with_wrong_treatment: [False]

patient_id: ['Helga_0 Root']
injury_id: ['L Stomach Puncture']
injury_required_procedure: ['woundpack']
injury_record_required_procedure: ['woundpack']
injury_treated_required_procedure: ['woundpack']
injury_treated_injury_treated_with_wrong_treatment: [False]

patient_id: ['Helga_0 Root']
injury_id: ['L Thigh Puncture']
injury_required_procedure: ['tourniquet']
injury_record_required_procedure: ['tourniquet']
injury_treated_required_procedure: ['tourniquet']
injury_treated_injury_treated_with_wrong_treatment: [True]

patient_id: ['Lily_1 Root']
injury_id: ['R Bicep Puncture']
injury_required_procedure: ['tourniquet']
injury_record_required_procedure: ['tourniquet']
injury_treated_required_procedure: ['tourniquet']
injury_tr

In [66]:

for (session_uuid, scene_id, patient_id), patient_df in merge_df[mask_series].groupby(fu.patient_groupby_columns):
    
    # Get the entire patient history
    patient_mask_series = True
    for cn in fu.patient_groupby_columns: patient_mask_series &= (merge_df[cn] == eval(cn))
    patient_df = merge_df[patient_mask_series]
    
    if (patient_df.shape[0] >= 5): break

In [29]:

from pandas import Series, concat

for responder_category, responder_categories_df in elevens_df.groupby('responder_category'):
    if (responder_category != 'Nurse or Other HP'):
        describable_df = concat([
            Series(
                [fu.get_triage_time(scene_df) for _, scene_df in responder_categories_df.groupby(fu.scene_groupby_columns)]
            ).to_frame().rename(columns={0: 'triage_time'}),
            Series(
                [fu.get_time_to_last_hemorrhage_controlled(scene_df) for _, scene_df in responder_categories_df.groupby(fu.scene_groupby_columns)]
            ).to_frame().rename(columns={0: 'last_controlled_time'})
        ], axis='columns')
        break

In [34]:

df = describable_df.sample(5)
print(f"""
        # Create a sample DataFrame
        self.describable_df = DataFrame({{
            'triage_time': {df.triage_time.tolist()},
            'last_controlled_time': {df.last_controlled_time.tolist()}
        }})""")


        # Create a sample DataFrame
        self.describable_df = DataFrame({
            'triage_time': [461207, 615663, 649185, 19615, 488626],
            'last_controlled_time': [377263, 574558, 462280, 0, 321956]
        })


In [37]:

# Create a sample DataFrame
describable_df = DataFrame({
    'triage_time': [461207, 615663, 649185, 19615, 488626],
    'last_controlled_time': [377263, 574558, 462280, 0, 321956]
})
expected_df = DataFrame(
    {
        'triage_time': [446859.2, 19615.0, 488626.0, 251951.57452415334, 19615.0, 461207.0, 488626.0, 615663.0, 649185.0],
        'last_controlled_time': [347211.4, 0.0, 377263.0, 216231.3283379631, 0.0, 321956.0, 377263.0, 462280.0, 574558.0]
    },
    index=['mean', 'mode', 'median', 'SD', 'min', '25%', '50%', '75%', 'max']
)

# Test with all columns
columns_list = describable_df.columns.tolist()
actual_df = fu.get_statistics(describable_df, columns_list)
actual_df.to_dict(orient='list')

{'triage_time': [446859.2, 19615.0, 488626.0, 251951.57452415334, 19615.0, 461207.0, 488626.0, 615663.0, 649185.0], 'last_controlled_time': [347211.4, 0.0, 377263.0, 216231.3283379631, 0.0, 321956.0, 377263.0, 462280.0, 574558.0]}

In [6]:

for (session_uuid, scene_id), scene_df in elevens_df.groupby(fu.scene_groupby_columns):
    if scene_df.shape[0] >= 2: break
df = scene_df.sample(3)
print(f"""
        # Create a mock DataFrame with scene start and end time columns
        self.scene_df = DataFrame({{
            "action_tick": {df.action_tick.tolist()}
        }})""")


        # Create a mock DataFrame with scene start and end time columns
        self.scene_df = pd.DataFrame({
            "action_tick": [575896, 693191, 699598]
        })


In [9]:

# Create a mock DataFrame with scene start and end time columns
from pandas import DataFrame
scene_df = DataFrame({
    "action_tick": [575896, 693191, 699598]
})
fu.get_triage_time(scene_df)

123702

In [3]:

# Example usage
zero_seconds_s = fu.format_timedelta(timedelta(seconds=0), minimum_unit='seconds')
zero_seconds_m = fu.format_timedelta(timedelta(seconds=0), minimum_unit='minutes')
thirty_seconds_s = fu.format_timedelta(timedelta(seconds=30), minimum_unit='seconds')
thirty_seconds_m = fu.format_timedelta(timedelta(seconds=30), minimum_unit='minutes')
one_minute_s = fu.format_timedelta(timedelta(minutes=1, seconds=0), minimum_unit='seconds')
one_minute_m = fu.format_timedelta(timedelta(minutes=1, seconds=0), minimum_unit='minutes')
a_minute_and_a_half_s = fu.format_timedelta(timedelta(minutes=1, seconds=30), minimum_unit='seconds')
a_minute_and_a_half_m = fu.format_timedelta(timedelta(minutes=1, seconds=30), minimum_unit='minutes')
two_minute_s = fu.format_timedelta(timedelta(minutes=2, seconds=0), minimum_unit='seconds')
two_minute_m = fu.format_timedelta(timedelta(minutes=2, seconds=0), minimum_unit='minutes')

print(
    f"zero seconds: '{zero_seconds_s}' or '{zero_seconds_m}'\n"
    f"thirty seconds: '{thirty_seconds_s}' or '{thirty_seconds_m}'\n"
    f"one minute: '{one_minute_s}' or '{one_minute_m}'\n"
    f"a minute and a half: '{a_minute_and_a_half_s}' or '{a_minute_and_a_half_m}'\n"
    f"two minutes: '{two_minute_s}' or '{two_minute_m}'"
)

zero seconds: '0 sec' or '00:00'
thirty seconds: '30 sec' or '00:30'
one minute: '1 min' or '01:00'
a minute and a half: '1:30' or '01:30'
two minutes: '2 min' or '02:00'


In [48]:

mask_series = ~elevens_df.injury_required_procedure.isnull()
for (session_uuid, scene_id, patient_id), patient_df in elevens_df[mask_series].groupby(fu.patient_groupby_columns):
    if patient_df.shape[0] >= 3:
        
        # Get the entire history of this patient
        mask_series = True
        for cn in fu.patient_groupby_columns: mask_series &= (elevens_df[cn] == eval(cn))
        patient_df = elevens_df[mask_series]
        
        mask_series = ~patient_df.injury_record_required_procedure.isnull()
        record_count = patient_df[mask_series].shape[0]
        mask_series = ~patient_df.injury_treated_required_procedure.isnull()
        treated_count = patient_df[mask_series].shape[0]
        if (record_count > 1) and (treated_count > 1): break

In [52]:

mask_series = ~patient_df.injury_required_procedure.isnull()
df = patient_df[mask_series].sample(3)
print(f"""
        # Create mock data for the patient DataFrame
        self.patient_df = DataFrame({{
            'action_type': {df.action_type.tolist()},
            'action_tick': {df.action_tick.tolist()},
            'injury_id': {df.injury_id.tolist()},
            'injury_record_required_procedure': {df.injury_record_required_procedure.tolist()},
            'injury_treated_required_procedure': {df.injury_treated_required_procedure.tolist()}
        }})""")


        # Create mock data for the patient DataFrame
        self.patient_df = DataFrame({
            'action_type': ['INJURY_TREATED', 'INJURY_TREATED', 'INJURY_RECORD'],
            'action_tick': [455202, 448909, 305828],
            'injury_id': ['R Side Puncture', 'R Thigh Laceration', 'R Side Puncture'],
            'injury_record_required_procedure': [nan, nan, 'woundpack'],
            'injury_treated_required_procedure': ['woundpack', 'gauzePressure', nan]
        })


In [53]:

fu.responder_negotiations_list

['PULSE_TAKEN', 'PATIENT_ENGAGED', 'INJURY_TREATED', 'TAG_APPLIED', 'TOOL_APPLIED']

In [55]:

# Create mock data for the patient DataFrame
from numpy import nan

# Create mock data for the patient DataFrame
patient_df = DataFrame({
    'action_type': ['INJURY_TREATED', 'INJURY_TREATED', 'INJURY_RECORD'],
    'action_tick': [455202, 448909, 305828],
    'injury_id': ['R Side Puncture', 'R Thigh Laceration', 'R Side Puncture'],
    'injury_record_required_procedure': [nan, nan, 'woundpack'],
    'injury_treated_required_procedure': ['woundpack', 'gauzePressure', nan]
})
fu.get_time_to_hemorrhage_control(patient_df, verbose=False)

6293

In [56]:

scene_start = 300_000
fu.get_time_to_hemorrhage_control(patient_df, scene_start=scene_start, verbose=False)

155202

In [ ]:

functions_list = [
    'colors_category_order', 'error_table_df', 'errors_category_order', 'format_timedelta', 'get_elevens_data_frame', 'get_last_tag',
    'get_max_salt', 'get_percent_hemorrhage_controlled', 'get_statistics', 'get_time_to_hemorrhage_control',
    'get_time_to_last_hemorrhage_controlled', 'get_triage_time', 'hemorrhage_control_procedures_list', 'is_patient_hemorrhaging',
    'patient_groupby_columns', 'salt_category_order', 'salt_to_tag_dict', 'scene_groupby_columns', 'tag_colors'
]
function_regex = re.compile(r'\b(' + '|'.join(functions_list) + r')\b')

In [4]:

# Create another test suite
suite = unittest.TestSuite()

# Add all test cases from the class to the suite
from test_color_functions import TestColorDistanceFrom, TestGetTextColor
suite.addTest(unittest.makeSuite(TestColorDistanceFrom))
suite.addTest(unittest.makeSuite(TestGetTextColor))

# Run the test suite
runner = unittest.TextTestRunner()
runner.run(suite)

......
----------------------------------------------------------------------
Ran 6 tests in 0.008s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>